## Preprocessing

In [103]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [104]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(["EIN","NAME","USE_CASE"], 1)

<ipython-input-104-cb6cc283a147>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  application_df = application_df.drop(["EIN","NAME","USE_CASE"], 1)


In [105]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [106]:
# Look at APPLICATION_TYPE value counts for binning
a_counts = application_df["APPLICATION_TYPE"].value_counts()
a_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [107]:
list([a_counts[a_counts < 500]])

[T9     156
 T13     66
 T12     27
 T2      16
 T25      3
 T14      3
 T29      2
 T15      2
 T17      1
 Name: APPLICATION_TYPE, dtype: int64]

In [108]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(a_counts[a_counts < 500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [109]:
# Look at CLASSIFICATION value counts for binning
c_counts = application_df["CLASSIFICATION"].value_counts()
c_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [110]:
from os import cpu_count
# You may find it helpful to look at CLASSIFICATION value counts >1
c_counts[c_counts > 500]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Name: CLASSIFICATION, dtype: int64

In [111]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(c_counts[c_counts < 500].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [112]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df)
application_df

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [113]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop(["IS_SUCCESSFUL"], axis='columns').values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=48)

In [114]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [115]:
len(X_train[0])

39

## Compile, Train and Evaluate the Model

In [116]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
num_input = len(X_train[0])
h1_1 = 10
h1_2 = 15
h1_3 = 10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=h1_1, input_dim=num_input, activation="relu")
)
# Second hidden layer
nn.add(
    tf.keras.layers.Dense(units=h1_2, activation="relu")
)

#Third hidden layer
nn.add(
    tf.keras.layers.Dense(units=h1_3, activation="relu")
)
# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_36 (Dense)            (None, 10)                400       
                                                                 
 dense_37 (Dense)            (None, 15)                165       
                                                                 
 dense_38 (Dense)            (None, 10)                160       
                                                                 
 dense_39 (Dense)            (None, 1)                 11        
                                                                 
Total params: 736 (2.88 KB)
Trainable params: 736 (2.88 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [117]:
# Compile the model
nn.compile( loss="binary_crossentropy", optimizer ="adam", metrics=["accuracy"])

In [118]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=60)

Epoch 1/60
804/804 [==============================] - 2s 2ms/step - loss: 0.5892 - accuracy: 0.7137
Epoch 2/60
804/804 [==============================] - 1s 2ms/step - loss: 0.5593 - accuracy: 0.7274
Epoch 3/60
804/804 [==============================] - 1s 2ms/step - loss: 0.5563 - accuracy: 0.7295
Epoch 4/60
804/804 [==============================] - 1s 2ms/step - loss: 0.5547 - accuracy: 0.7301
Epoch 5/60
804/804 [==============================] - 1s 2ms/step - loss: 0.5533 - accuracy: 0.7316
Epoch 6/60
804/804 [==============================] - 1s 2ms/step - loss: 0.5527 - accuracy: 0.7315
Epoch 7/60
804/804 [==============================] - 2s 2ms/step - loss: 0.5521 - accuracy: 0.7308
Epoch 8/60
804/804 [==============================] - 2s 2ms/step - loss: 0.5511 - accuracy: 0.7304
Epoch 9/60
804/804 [==============================] - 1s 2ms/step - loss: 0.5511 - accuracy: 0.7315
Epoch 10/60
804/804 [==============================] - 1s 2ms/step - loss: 0.5505 - accuracy: 0.7317

In [119]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5544 - accuracy: 0.7291 - 436ms/epoch - 2ms/step
Loss: 0.5543919801712036, Accuracy: 0.7290962338447571


In [120]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
